<a href="https://colab.research.google.com/github/Exter-dg/Scheduling-Algorithms/blob/master/OS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#1 FCFS
#2 
#3 Round-Robin


def waitingTime(size,bt,at,wt):
  sumTime = [0] * size
  
  for i in range(1,size):

    sumTime[i] = sumTime[i-1] + bt[i-1]
    wt[i] = sumTime[i] - at[i]

    # when it arrives late and cpu is idle
    if(wt[i] < 0):
      wt[i] = 0

def turnAroundTime(size,bt,wt,tat):
  
  for i in range(0,size):
    tat[i] = wt[i] + bt[i]


def avgTime(processes,size,bt,at):
  wt = [0] * size
  tat = [0] * size
  ct = [0] * size

  waitingTime(size,bt,at,wt)
  turnAroundTime(size,bt,wt,tat)

  totalWT=0
  totalTAT=0

  for i in range(0,size):
    totalTAT = totalTAT + tat[i]
    totalWT = totaltWT + wt[i]
    ct[i] = tat[i] + at[i]

  #TODO HERE
  #print proper formatted table displaying all the info about fcfs
  


In [0]:
#2

In [0]:
#3 Round Robin